# Collect simulation data for calibration points

In [9]:
# Setup ipython environment
%load_ext autoreload
%autoreload 2
%matplotlib inline
from matplotlib.pyplot import *
from numpy import *
from positive import *
from nrutils import scsearch, gwylm
from glob import glob
import h5py

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
#
datadir = '/Users/book/KOALA/puck/ll/data/'
h5list = glob(datadir+'*.h5')

In [10]:
for h in h5list:
    print h
    f = h5py.File(h, 'r')
    break

/Users/book/KOALA/puck/ll/data/q1a04t150_dP0_T_96_552.h5


In [11]:
f.keys()

[u'psi4', u'strain', u'times']

In [14]:
plot(f['times'],f['strain'])

AttributeError: 'Dataset' object has no attribute 'real'